# MNIST Multilayer Perceptron

## Code setup

In [1]:
# Suppressing warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
# Utility imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
import gc
import pickle

# Source imports
from mlp_generator import one_layer, two_layers, three_layers, four_layers
from mlp_trainer import dataset_creation, train_MLP
from fault_simulation import run_simulation

In [3]:
# The statements below are employed to control the mode in which the Jupyter
# notebook operates.

# Legal values: [1, 2, 3, 4]
number_of_hidden_layers = 1

# Legal values: [1, 2, 3]; 1: Cannot electroform, 2: Stuck at HRS, 3: Stuck at LRS
fault_type = 2

# Device parameters
HRS_LRS_ratio = 5
excluded_weights_proportion = 0.015

## Neural network setup and training

In [4]:
# Create dataset and results lists
MNIST_dataset = dataset_creation()
weights_list = []
histories_list = []

# Helper values
generator_functions = {1: one_layer, 2:two_layers, 3:three_layers, 4:four_layers}
number_of_ANNs = 5

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
# Model definition and training, repeated 30 times to average out stochastic variancies
for model_number in range(0, number_of_ANNs):

    MNIST_MLP = generator_functions[number_of_hidden_layers]()
    MLP_weights, MLP_history, *_ = train_MLP(MNIST_dataset, MNIST_MLP, epochs=10, batch_size=100)
    weights_list.append(MLP_weights)
    histories_list.append(MLP_history)
    
    if (model_number+1) % 5 == 0 and model_number != 0:
        print("Model #{} finished training.".format(model_number+1))
    
    gc.collect()

Model #5 finished training.


## Performance evolution during training

In [6]:
# Computing training and validation loss and accuracy by averaging over all the models trained in the previous step

epochs = range(1, len(histories_list[0].history["accuracy"]) + 1)
accuracy_values = np.zeros(len(histories_list[0].history["accuracy"]))
validation_accuracy_values = np.zeros(len(histories_list[0].history["accuracy"]))
loss_values = np.zeros(len(histories_list[0].history["accuracy"]))
validation_loss_values = np.zeros(len(histories_list[0].history["accuracy"]))

for MLP_history in histories_list:

    history_dict = MLP_history.history
    accuracy_values += np.array(history_dict["accuracy"])
    validation_accuracy_values += np.array(history_dict["val_accuracy"])
    loss_values += np.array(history_dict["loss"])
    validation_loss_values += np.array(history_dict["val_loss"])

accuracy_values /= len(histories_list)
validation_accuracy_values /= len(histories_list)
loss_values /= len(histories_list)
validation_loss_values /= len(histories_list)

In [7]:
# Saving training/validation data to file
pickle.dump((accuracy_values, validation_accuracy_values, loss_values, validation_loss_values), open("saved_data/training_validation_{}HL.p".format(number_of_hidden_layers), "wb"))

## Fault simulation: Cannot electroform, Stuck at HRS, Stuck at LRS

In [8]:
# # Setting variable percentages of the ANN's synaptic weights to zero in order to simulate devices being unable to electroform

# percentages = np.arange(0, 1.01, 0.01)
# accuracies = np.zeros(len(percentages))

# MNIST_MLP = generator_functions[number_of_hidden_layers]()
# MNIST_MLP.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# for count, weights in enumerate(weights_list):

#     accuracies_list = []

#     for percentage in percentages:
        
#         altered_weights = alteration_functions[fault_type](weights, percentage)
        
#         # The "set_weights" function sets the ANN's weights to the values specified in the list of arrays "altered_weights"
#         MNIST_MLP.set_weights(altered_weights)
#         accuracies_list.append(MNIST_MLP.evaluate(MNIST_dataset[1][0], MNIST_dataset[1][1], verbose=0)[1])

#     accuracies += np.array(accuracies_list)

#     if (count+1) % 5 == 0 and count != 0:
#         print("Finished evaluating weight set #{}.".format(count+1))
    
#     gc.collect()

# accuracies /= len(weights_list)

# # Saving accuracies array to file
# pickle.dump((percentages, accuracies, fault_type), open("saved_data/accuracies_faultType{}_{}HL.p".format(fault_type, number_of_hidden_layers), "wb"))

In [9]:
# Running 30 simulations for each of the 30 networks trained above over the specified
# range of faulty devices percentages

MNIST_MLP = generator_functions[number_of_hidden_layers]()
MNIST_MLP.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

percentages = np.arange(0, 1.01, 0.01)
accuracies_array = np.zeros((len(weights_list), len(percentages)))

number_of_simulations = 5

for count, weights in enumerate(weights_list):

    accuracies_array[count] = run_simulation(percentages, weights, number_of_simulations, MNIST_MLP, MNIST_dataset, fault_type, HRS_LRS_ratio, excluded_weights_proportion)
    gc.collect()

    if (count+1) % 5 == 0 and count != 0:
        print("Finished evaluating weight set #{}.".format(count+1))

# Averaging the results obtained for each of the 30 sets of weights
accuracies = np.mean(accuracies_array, axis=0, dtype=np.float64)

# Saving accuracies array to file
pickle.dump((percentages, accuracies, fault_type), open("saved_data/accuracies_faultType{}_{}HL.p".format(fault_type, number_of_hidden_layers), "wb"))

Finished evaluating weight set #5.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f63981ff-15e3-4a0a-8bcf-dbdeefafcfde' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>